# Model serving & deployment testing

## Abstract

This notebook test model deployment using a pre-transformed example.

In [1]:
ENDPOINT_ID='5909171311854223360' 
GOOGLE_CLOUD_PROJECT = 'aliz-ml-spec-2022-submission'
GOOGLE_CLOUD_REGION = 'us-central1'

In [1]:
!gsutil cp -r gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220608212029/Transform_-1809382173191438336/transformed_examples transform/

Copying gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220608212029/Transform_-1809382173191438336/transformed_examples/Split-eval/transformed_examples-00000-of-00001.gz...
Copying gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220608212029/Transform_-1809382173191438336/transformed_examples/Split-test/transformed_examples-00000-of-00001.gz...
Copying gs://aliz-ml-spec-2022/demo-1/pipeline_root/taxi-vertex-pipelines/53911330556/taxi-vertex-pipelines-20220608212029/Transform_-1809382173191438336/transformed_examples/Split-train/transformed_examples-00000-of-00001.gz...
- [3 files][ 56.3 MiB/ 56.3 MiB]                                                
Operation completed over 3 objects/56.3 MiB.                                     


In [2]:
!gunzip transform/transformed_examples/Split-eval/transformed_examples-00000-of-00001.gz

In [7]:
from google.cloud import aiplatform
import tensorflow as tf

# The AI Platform services require regional API endpoints.
client_options = {
    'api_endpoint': GOOGLE_CLOUD_REGION + '-aiplatform.googleapis.com'
    }
# Initialize client that will be used to create and send requests.
client = aiplatform.gapic.PredictionServiceClient(client_options=client_options)


raw_dataset = tf.data.TFRecordDataset("transform/transformed_examples/Split-eval/transformed_examples-00000-of-00001")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())

instance_dict = {}
for feature_name in example.features.ListFields()[0][1]:
    if feature_name != 'fare':
        try:
            instance_dict[feature_name] = [example.features.ListFields()[0][1][feature_name].float_list.value[0]]
        except:
            instance_dict[feature_name] = [example.features.ListFields()[0][1][feature_name].int64_list.value[0]]

instances = [instance_dict]

endpoint = client.endpoint_path(
    project=GOOGLE_CLOUD_PROJECT,
    location=GOOGLE_CLOUD_REGION,
    endpoint=ENDPOINT_ID,
)
response = client.predict(endpoint=endpoint, instances=instances)
print('prediction:', response.predictions[0])

prediction: [27.9183121]


The prediction of 27.9 we get back is fairly close to the expected 37.5 from the example